In [ ]:
import os
os.listdir("testimages/")

In [ ]:
# Import everything needed to edit/save/watch video clips
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
#load relavent libraries
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

#define helper functions
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def hsv(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)

    if len(img.shape) > 2:
       channel_count = img.shape[2]
       ignore_mask_color = (255, ) * channel_count
    else:
       ignore_mask_color = 255

    cv2.fillPoly(mask, vertices, ignore_mask_color)

    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines):
    color = (255, 0, 0)
    thickness  = 10
 
    #principle: find the points which give the maximum length
    disTemp = 0
    startP = [0, 0]
    endP   = [0, 0]
    for line in lines:
        for x1, y1, x2, y2 in line:
            dis = (x2 - x1)**2 + (y2 - y1)**2
            if ((dis > disTemp) and (abs(x2-x1) > 1 and abs(y2-y1) > 1)):
                disTemp = dis
                startP[0] = x1
                startP[1] = y1
                endP[0]   = x2
                endP[1]   = y2
               
    slope  = (endP[1] - startP[1]) / (endP[0] - startP[0])
    pivot  = endP[1] - slope * endP[0]
    
    #create two arrays: xelement saves x axes of lines. yelement saves y axes
    i = 0
    yelement = np.zeros(2*(lines.shape)[0])
    xelement = np.zeros(2*(lines.shape)[0])
    for line in lines:
        for x1, y1, x2, y2 in line:
            yelement[i] = y1
            xelement[i] = x1
            i = i + 1
            yelement[i] = y2
            xelement[i] = x2
            i = i + 1
    
    #determine the starting and ending pixels. 
    ymax = (img.shape)[0]   
    xmax = (int)((ymax-pivot)/slope)
    
    if (np.max(xelement) <= round(img.shape[1]/2)):
        xmin = (int)(np.max(xelement))
    else:
        xmin = (int)(round(img.shape[1]/2))
    ymin = (int)(xmin*slope + pivot)
    
    #draw the fitting line
    cv2.line(img, (xmin, ymin), (xmax, ymax), color, thickness)
            
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),minLineLength = min_line_len, maxLineGap = max_line_gap)
    return lines

def weighted_img(img, initial_img, alfa, beta, lamda):
    return cv2.addWeighted(initial_img, alfa, img, beta, lamda)

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    hsv_img = hsv(image)
        
    # processing pipeline
    gray = grayscale(image)

    # define range of color in HSV
    lower_yel = np.array([20,100,100])
    upper_yel = np.array([30,255,255])
    lower_wht = np.array([0,0,235])
    upper_wht = np.array([255,255,255])
      
    # Create yellow mask an white mask 
    yellow_mask = cv2.inRange(hsv_img, lower_yel, upper_yel)
    white_mask = cv2.inRange(hsv_img, lower_wht, upper_wht)
        
    # Filter original image
    full_mask = cv2.bitwise_or(yellow_mask, white_mask)
    subdued_gray = (gray / 2).astype('uint8')
    enhance_lanes = cv2.bitwise_or(subdued_gray, full_mask)
    
    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(enhance_lanes,kernel_size)
        
    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 180
    edges = canny(blur_gray, low_threshold, high_threshold)
    
    # Next we'll create a masked edges image using cv2.fillPoly()
    imshape = image.shape
    line_image_left = np.copy(image)*0 # creating a blank to draw lines on        
    vertices_left = np.array([[(0, imshape[0]), (round(0.8*0.5*imshape[1]), round(imshape[0]/2)), (round(imshape[1]/2),round(imshape[0]/2)), (round(0.5*imshape[1]), imshape[0])]], dtype=np.int32)
    vertices_right = np.array([[ (round(imshape[1]/2), round(imshape[0]/2)), (round(imshape[1]/2), imshape[0]), (imshape[1], imshape[0]), (round(imshape[1]*0.5*1.2), round(imshape[0]*0.5))]], dtype=np.int32)
    masked_image_left = region_of_interest(edges, vertices_left)
    masked_image_right = region_of_interest(edges, vertices_right)

          
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 40     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #minimum number of pixels making up a line
    max_line_gap = 200    # maximum gap in pixels between connectable line segments
    line_image_left = np.copy(image)*0 # creating a blank to draw lines on
    line_image_right = np.copy(image)*0 # creating a blank to draw lines on
        
    # Run Hough on edge detected image 
    # Output "lines" is an array containing endpoints of detected line segments
    lines_left = hough_lines(masked_image_left, rho, theta, threshold,
                 min_line_length, max_line_gap)
    lines_right = hough_lines(masked_image_right, rho, theta, threshold,
                  min_line_length, max_line_gap)

    # Draw lines on a blank image
    draw_lines(line_image_left, lines_left)
    draw_lines(line_image_right, lines_right)
        
    line_image = np.copy(image)*0
    line_image = line_image_left + line_image_right
        
    # Create a "color" binary image to combine with line image 
    color_edges = np.dstack((edges, edges, edges)) 
        
    # Draw the lines on the edge image
    result = weighted_img(image, line_image, 0.8, 1, 0) 
        
    return result

In [ ]:
white_output = 'testimages/white.mp4'
clip2 = VideoFileClip('testimages/solidWhiteRight.mp4')
white_clip = clip2.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_clip))

In [ ]:
yellow_output = 'testimages/yellow.mp4'
clip2 = VideoFileClip('testimages/solidYellowRight.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_clip))

In [ ]:
challenge_output = 'testimages/challengeOutput.mp4'
clip2 = VideoFileClip('testimages/challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_clip))